# Particle Swarm Optimiser

In [10]:
import os, sys, json 
import numpy as np
sys.path.append(os.path.abspath(".."))

from velopix_wrappers.parameter_optimisers import optimiserBase
from velopix_wrappers.velopix_pipeline import TrackFollowingPipeline, GraphDFSPipeline, SearchByTripletTriePipeline

## Implement the optimiser child class

In [11]:
from copy import deepcopy
import random 

class ParticleSwarm(optimiserBase):
    def __init__(self,
        swarm_size: int = 10,
        w: float = 0.5,
        c1: float = 1.5,
        c2: float = 1.5,
        convergence_tolerance: float = 1e-4,
        patience: int = 15,
        **kwargs):
        super().__init__(Objective="min")
        self.swarm_size = swarm_size
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.convergence_tolerance = convergence_tolerance
        self.patience = patience

        # These attributes will be set during init()
        self.swarm = []         # List of particle positions (each a dict)
        self.velocities = []    # List of particle velocities (dicts matching the particle structure)
        self.pbest = []         # Personal best positions for each particle
        self.pbest_scores = []  # Best scores for each particle
        self.iterations = 0
        self.current_particle_index = 0  # Which particle's turn to be updated/evaluated
        self.score_history = []

    def is_finished(self):
        if len(self.score_history) > self.patience:
            recent_scores = self.score_history[-self.patience:]
            improvement = max(recent_scores) - min(recent_scores)
            if improvement < self.convergence_tolerance:
                return True
        return False
    
    def init(self):
        schema = self._algorithm.get_config() # get the schema for the track reconstruction algo 

        self.swarm = []
        self.velocities = []
        self.pbest = []
        self.pbest_scores = []
        self.score_history = []

        for _ in range(self.swarm_size):
            particle = {}
            velocity = {}
            for key, (expected_type, _) in schema.items():
                # init bools at random
                if expected_type == bool:
                    particle[key] = random.choice([True, False])
                    velocity[key] = 0.0
                elif expected_type == float or expected_type == int:
                    low, high = self._algorithm._bounds().get(key)
                    particle[key] = expected_type(random.uniform(low, high))
                velocity[key] = 0.0

            self.swarm.append(particle)
            self.velocities.append(velocity)
            self.pbest.append(deepcopy(particle))
            self.pbest_scores.append(float("inf"))

        self.best_score = float("inf")
        self.best_config = deepcopy(self.swarm[0])
        self.iterations = 0
        self.current_particle_index = 0
        return deepcopy(self.swarm[0])
    
    def next(self):
        # Cycle to the next particle in the swarm, since we need to eval each particle at the time in the algo's
        self.current_particle_index = (self.current_particle_index + 1) % self.swarm_size
        idx = self.current_particle_index
        schema = self._algorithm.get_config()

        for key, (expected_type, _) in schema.items():
            if expected_type == bool:
                # note sure what to do here yet
                continue 

            low, high = self._algorithm._bounds().get(key)
            r1 = random.random()
            r2 = random.random()
            current_vel = self.velocities[idx][key]
            current_pos = self.swarm[idx][key]
            pbest_pos = self.pbest[idx][key]
            gbest_contrib = 0.0 if self.best_score == float("inf") else self.best_config[key] - current_pos

            new_vel = (
                self.w * current_vel +
                self.c1 * r1 * (pbest_pos - current_pos) +
                self.c2 * r2 * gbest_contrib
            )
            self.velocities[idx][key] = new_vel

            new_pos = current_pos + new_vel
            new_pos = max(low, min(new_pos, high))
            self.swarm[idx][key] = expected_type(new_pos)

        self.iterations += 1
        candidate = deepcopy(self.swarm[idx])

        candidate_score = self.objective_func() # We're minimising the score
        idx = self.current_particle_index
        self.score_history.append(candidate_score)

        if candidate_score < self.pbest_scores[idx]:
            self.pbest_scores[idx] = candidate_score
            self.pbest[idx] = deepcopy(self.swarm[idx])

        if candidate_score < self.best_score:
            self.best_score = candidate_score
            self.best_config = deepcopy(self.swarm[idx])

        return candidate
    
    def objective_func(self): # we can generate custom objective functions here or use a standard defined objective function 
        return self.event_objective()

**Load event data**

In [12]:
events = []
n_files = 100

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data)
        event_file.close()

Loading file: velo_event_0.json
Loading file: velo_event_1.json
Loading file: velo_event_2.json
Loading file: velo_event_3.json
Loading file: velo_event_4.json
Loading file: velo_event_5.json
Loading file: velo_event_6.json
Loading file: velo_event_7.json
Loading file: velo_event_8.json
Loading file: velo_event_9.json
Loading file: velo_event_10.json
Loading file: velo_event_11.json
Loading file: velo_event_12.json
Loading file: velo_event_13.json
Loading file: velo_event_14.json
Loading file: velo_event_15.json
Loading file: velo_event_16.json
Loading file: velo_event_17.json
Loading file: velo_event_18.json
Loading file: velo_event_19.json
Loading file: velo_event_20.json
Loading file: velo_event_21.json
Loading file: velo_event_22.json
Loading file: velo_event_23.json
Loading file: velo_event_24.json
Loading file: velo_event_25.json
Loading file: velo_event_26.json
Loading file: velo_event_27.json
Loading file: velo_event_28.json
Loading file: velo_event_29.json
Loading file: velo_e

In [13]:
TF_pipeline = TrackFollowingPipeline(events=events, intra_node=False)
Graph_pipeline = GraphDFSPipeline(events=events, intra_node=False)
Triplet_pipeline = SearchByTripletTriePipeline(events=events, intra_node=False)

In [14]:
Optimiser = ParticleSwarm(
    swarm_size=2,
    w=0.1,
    c1=0.25,
    c2=0.3,
    convergence_tolerance=1e-3,
    patience=7
) 

In [15]:
optimal_parameters_TF = TF_pipeline.optimise_parameters(Optimiser, max_runs=10) # DO NOT remove max_runs, chances are that this will run forever

Optimising: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


In [16]:
optimal_parameters_Graph = Graph_pipeline.optimise_parameters(Optimiser, max_runs=10)

: 

: 

In [ ]:
optimal_parameters_Triplet = Triplet_pipeline.optimise_parameters(Optimiser, max_runs=10)

Optimising: 100%|██████████| 10/10 [00:13<00:00,  1.39s/it]


In [ ]:
print(Optimiser.get_optimised_pMap())

In [ ]:
print(optimal_parameters_TF)

{'x_slope': 70.40756777753698, 'y_slope': 33.813617985698905, 'x_tol': 1.8339073102118497, 'y_tol': 5.974312516342821, 'scatter': 3.5862934522445915}
